# YAHOO電影爬蟲練習
## 練習爬取電影放映資訊。必須逐步獲取電影的代號、放映地區、放映日期後，再送出查詢給伺服器。

In [2]:
import requests
import re
from bs4 import BeautifulSoup
import json

### 先搜尋全部的電影代號(ID)資訊

In [1]:
# 查看目前上映那些電影，並擷取出其ID資訊
url = 'https://movies.yahoo.com.tw/'
resp = requests.get(url)
soup = BeautifulSoup(resp.text, 'html5lib')
select = soup.find('select', attrs = {'name':'movie_id'})
id_info = select.find_all('option', attrs = {'data-name': re.compile('.*')})
for info in id_info:
    print('Movie: %s, ID: %s' % (info.text, info['value']))

Movie: 白頭山：半島浩劫, ID: 10354
Movie: CATS貓, ID: 10166
Movie: 海上花, ID: 10376
Movie: 奪魂露營車, ID: 10366
Movie: 世紀球王 馬拉度納, ID: 10361
Movie: 怨靈古堡, ID: 10356
Movie: 添好孕, ID: 10332
Movie: 外出偷馬, ID: 10310
Movie: 茱蒂, ID: 10112
Movie: 燃燒女子的畫像, ID: 10069
Movie: 葉問4：完結篇, ID: 9890
Movie: STAR WARS : 天行者的崛起, ID: 10225
Movie: 復仇母親, ID: 10392
Movie: 風暴過後, ID: 10387
Movie: 南方車站的聚會, ID: 10380
Movie: 祭旗, ID: 10373
Movie: 唯你是愛, ID: 10370
Movie: 屍控療法, ID: 10367
Movie: 黑熊來了, ID: 10357
Movie: 唐人街探案2, ID: 10353
Movie: 黑計畫, ID: 10351
Movie: 假面騎士劇場版ZI-O Over Quartzer, ID: 10331
Movie: 巴黎小情聖, ID: 10329
Movie: 8級警戒, ID: 10323
Movie: 為你存在的每一天, ID: 10297
Movie: 黑色聖誕節, ID: 10291
Movie: 熱氣球飛行家, ID: 10257
Movie: 大說謊家, ID: 10122
Movie: 特約經紀公司, ID: 10364
Movie: 我的靈魂是愛做的, ID: 10336
Movie: 絕世情歌, ID: 10328
Movie: 人間失格：太宰治與他的3個女人, ID: 10287
Movie: 呆萌特務, ID: 10264
Movie: 去年聖誕節, ID: 10256
Movie: 野蠻遊戲：全面晉級, ID: 10134
Movie: 夕霧花園, ID: 10327
Movie: 布魯克林孤兒, ID: 10224
Movie: 暴走曼哈頓, ID: 10044
Movie: 鋒迴路轉, ID: 10258
Movie: 賽道狂人, ID: 100

### 指定你有興趣的電影其ID，然後查詢其放映地區資訊。

In [3]:
# 參考前一個步驟中擷取到的ID資訊，並指定ID
movie_id = 9597

In [4]:
url2 = 'https://movies.yahoo.com.tw/api/v1/areas_by_movie_theater'
pl = {'movie_id': str(movie_id)}
hds =  {
    'authority': 'movies.yahoo.com.tw',
    'method': 'GET',
    'path': '/api/v1/areas_by_movie_theater?movie_id=' + str(movie_id),
    'scheme': 'https',
    'accept': 'application/json, text/javascript, */*; q=0.01',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6',
    'cookie': 'rxx=9s3x2fws06.1g16irnc&v=1; _ga=GA1.3.2056742944.1551651301; GUC=AQEBAQFczFpdm0IfmwSB&s=AQAAACoo4N5D&g=XMsVBw; BX=4hkdk1decm57t&b=3&s=mr; _ga=GA1.4.2056742944.1551651301; nexagesuid=82843256dd234e8e91aa73f2062f8218; browsed_movie=eyJpdiI6IlJXWWtiSWJaZlNGK2MxQnhscnVUYWc9PSIsInZhbHVlIjoiMXRhMmVHRXRIeUNjc1RBWDJzdGYwbnlIQURmWGsrcjJSMzhkbkcraDNJVUNIZEZsbzU3amlFcVZ1NzlmazJrTGpoMjVrbHk1YmpoRENXaHZTOUw1TmI2ZTZVWHdOejZQZm16RmVuMWlHTTJLaTZLVFZZVkFOMDlTd1wvSGltcytJIiwibWFjIjoiZWQ2ZjA4MmVjZmZlYjlmNjJmYmY2NGMyMDI0Njc0NWViYjVkOWE2NDg0N2RhODMxZjBjZDhiMmJhZTc2MDZhYiJ9; avi=eyJpdiI6Im1NeWFJRlVRWDR1endEcGRGUGJUbVE9PSIsInZhbHVlIjoickRpU3JuUytmcGl6cjF5OW0rNU9iZz09IiwibWFjIjoiY2VmY2NkNzZmM2NhNjY5YzlkOTcyNjE5OGEyMzU0NWYxOTdmMDRkMDY3OWNmMmZjOTMxYjc5MjI5N2Q5NGE5MiJ9; cmp=t=1559391030&j=0; _gid=GA1.4.779543841.1559391031; XSRF-TOKEN=eyJpdiI6IkhpS2hGcDRQaHlmWUJmaHdSS2Q2bHc9PSIsInZhbHVlIjoiOUVoNFk4OHI1UUZmUWRtYXhza0MyWjJSTlhlZ3RnT0VGeVJPN2JuczVRMGRFdWt2OUlsamVKeHRobFwvcHBGM0dhU3VyMXNGTHlsb2dVM2l0U1hpUGxBPT0iLCJtYWMiOiJkZWU4YzJhNjAxMTY3MzE4Y2ExNWIxYmE1ZjE1YWZlZTlhOTcyYjc4M2RlZGY4ZWNjZDYyMTA2NGYwZGViMzc2In0%3D; m_s=eyJpdiI6InpsZHZ2Tk1BZ0dxaHhETml1RjBnUXc9PSIsInZhbHVlIjoiSkNGeHUranRoXC85bDFiaDhySTJqNkJRcWdjWUxjeVRJSHVYZ1wvd2d4bWJZUTUrSHVDM0lUcW5KNHdETFZ4T1lieU81OUhzc1VoUXhZcWk0UDZSQXVFdz09IiwibWFjIjoiYmJkMDJkMDhlODIzMzcyMWY4M2NmYWNjNGVlOWRjMDIwZmVmNzAyMjE3Yzg3ZGY3ODBkZWEzZTI4MTI5ZWNmOSJ9; _gat=1; nexagesd=10',
    'dnt': '1',
    'mv-authorization': '21835b082e15b91a69b3851eec7b31b82ce82afb',
    'referer': 'https://movies.yahoo.com.tw/',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36',
    'x-requested-with': 'XMLHttpRequest',
}
resp = requests.get(url2, params = pl, headers = hds)
import json
js = json.loads(resp.text)
for j in js:
    print('放映地區：{0}, 代號(area_id)：{1}'.format(j['title'], j['area_id']))

放映地區：台北市, 代號(area_id)：28
放映地區：新北市, 代號(area_id)：8
放映地區：桃園, 代號(area_id)：16
放映地區：新竹, 代號(area_id)：20
放映地區：苗栗, 代號(area_id)：15
放映地區：台中, 代號(area_id)：2
放映地區：彰化, 代號(area_id)：22
放映地區：南投, 代號(area_id)：13
放映地區：雲林, 代號(area_id)：19
放映地區：嘉義, 代號(area_id)：21
放映地區：台南, 代號(area_id)：10
放映地區：高雄, 代號(area_id)：17
放映地區：基隆, 代號(area_id)：18
放映地區：宜蘭, 代號(area_id)：11
放映地區：花蓮, 代號(area_id)：12
放映地區：台東, 代號(area_id)：9
放映地區：澎湖, 代號(area_id)：23


### 指定你想要觀看的放映地區，查詢有上映電影的場次日期

In [6]:
# 指定放映地區
area_id = 20 #新竹的播映時間

In [8]:
url3 = 'https://movies.yahoo.com.tw/movietime_result.html'
pl3 = {'movie_id': str(movie_id), 'area_id': str(area_id)}
resp3 = requests.get(url3, params=pl3)
soup3 = BeautifulSoup(resp3.text, 'html5lib')
times = soup3.find_all('label', class_="picker_label")
for t in times:
    print(t.find(class_='month').text+' '+t.find(class_='day').text+' '+t.find(class_='week').text)

十二月 24 今天
十二月 25 明天
十二月 26 禮拜四
十二月 27 禮拜五
十二月 28 禮拜六


### 最後指定觀看的日期，查詢並列印出放映的電影院、放映類型(數位、3D、IMAX 3D...)、放映時間等資訊。

In [9]:
# 選定要觀看的日期
date = "2019-12-25"

In [10]:
date = '2019-12-25'
url4 = "https://movies.yahoo.com.tw/ajax/pc/get_schedule_by_movie"
pl4 = {'movie_id':str(movie_id),
           'date':date,
           'area_id':str(area_id),
           'theater_id':'',
           'datetime':'',
           'movie_type_id':''}
resp4 = requests.get(url4, pl4)
js4 = json.loads(resp4.text)
soup4 = BeautifulSoup(js4['view'], 'html5lib')

In [14]:
area = soup4.find_all('ul', class_='area_time')
for a in area:
    print('-----------------------------------------------------------------')
    print(a['data-theater_name'])
    print('電影名稱： {}'.format(a.find('input', class_='gabtn')['data-movie_title']))
    times = a.find_all('input', class_='gabtn')
    for b in times:
        print('放映類型： {}'.format(b['data-movie_type']))
        print(b['value'])
print('-----------------------------------------------------------------')

-----------------------------------------------------------------
新竹大遠百威秀影城
電影名稱： 冰雪奇緣2
放映類型： 數位
2019-12-25 10:35:00
放映類型： 數位
2019-12-25 14:25:00
放映類型： 數位
2019-12-25 18:25:00
-----------------------------------------------------------------
新竹巨城威秀影城
電影名稱： 冰雪奇緣2
放映類型： 數位
2019-12-25 09:25:00
放映類型： 數位
2019-12-25 13:35:00
放映類型： 數位
2019-12-25 17:35:00
-----------------------------------------------------------------
